

<br><br>

<h1><center>Applied Data Science Capstone Course</center></h1>

## Week 3 Assignment: 'Segmenting and Clustering Neighborhoods in Toronto'

## Author: Diego Medeiros

### GitHub: https://github.com/medeirox


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Data Scraping of Toronto's Neighborhood Data</a>

2. <a href="#item2">Collecting Longitude and Latitude from FourSquare</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

</font>
</div>

<a id='item1'></a>

## Data Scraping of Toronto's Neighborhood Data

First we'll collect data from Wikipedia (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) and process it to fit in a DataFrame using Pandas

In [103]:
import pandas as pd

In [104]:
d = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [105]:
df = d[0]
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


### Assign Borough to Neighbourhoods cointaining "Not assigned" information and drop rows with "Not assigned" Borough

In [106]:
df[df['Neighbourhood'].str.contains('Not assigned')] = df[['Postcode','Borough','Borough']]
df = df[df['Borough']!='Not assigned']
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


 ### Process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [107]:
df_clean = df[df['Borough'] != 'Not assigned']
df_clean.columns = ['PostalCode', 'Borough', 'Neighbourhood']
df_clean.head(10)

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


### Aggregate by Postcode and Borough, appending all Neighbourhoods from a given Borough in a single row

In [108]:
pc_groups = df_clean.groupby(['PostalCode', 'Borough']).agg(lambda x: ', '.join(x))
pc_groups.reset_index(inplace=True)
pc_groups.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Checking DataFrame information

In [109]:
pc_groups.shape

(103, 3)

In [110]:
pc_groups.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


In [111]:
pc_groups.describe()

,PostalCode,Borough,Neighbourhood
count,103,103,103
unique,103,11,103
top,M5G,North York,"The Annex, North Midtown, Yorkville"
freq,1,24,1


<a id='item2'></a>

## Collecting Longitude and Latitude from FourSquare

In [112]:
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
#!conda install -c conda-forge folium --yes
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Initialize Nominatim to collect Latitude and Longitude from locations

In [113]:
# Due to problems using the GeoPy package, I'll use the csv provided by the course
!wget -q -O 'toronto_postalcodes.csv' https://cocl.us/Geospatial_data

In [114]:
df_postalcodes = pd.read_csv('toronto_postalcodes.csv')
df_postalcodes.set_index('Postal Code', inplace=True)
df_postalcodes.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [115]:
for pc in pc_groups['PostalCode']:
    df_postalcodes.loc[pc]

In [116]:
import time
city = 'Toronto'
state = 'ON'

address = '{ct}, {st}'.format(ct=city, st=state)

# Geolocator not working...
#geolocator = Nominatim(user_agent="toronto_explorer")

df_latlon = pd.DataFrame(columns=['Latitude','Longitude'])

for pc in pc_groups['PostalCode']:
    df_latlon=df_latlon.append(df_postalcodes.loc[pc])
    
#print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))
df_latlon.head()

,Latitude,Longitude
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [117]:
print('Shape of df_latlon: {}'.format(df_latlon.shape))
print('Shape of pc_groups: {}'.format(pc_groups.shape))

Shape of df_latlon: (103, 2)
Shape of pc_groups: (103, 3)


In [118]:
df_latlon.reset_index(inplace=True)
pc_groups = pd.concat([pc_groups, df_latlon], axis=1)

In [120]:
pc_groups.head()

,PostalCode,Borough,Neighbourhood,index,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
